In [ ]:
#==============================================================================================================================================================
# INSTANCIATION DU SERVEUR AMAZON ELASTIC MAPREDUCE (EMR)
#==============================================================================================================================================================

# Amazon EMR : 6.10.0
# Hadoop 3.3.3
# Spark 3.3.1
# TensorFlow 2.11.0
# Ganglia 3.7.2
# JupyterHub 1.5.0
# Python 3.6

#EEEEEEEEEEEEEEEEEEE MMMMMMMM           MMMMMMMM RRRRRRRRRRRRRRR    
#::::::::::::::::::E M:::::::M         M:::::::M R::::::::::::::R   
#E:::::EEEEEEEEE:::E M::::::::M       M::::::::M R:::::RRRRRR:::::R 
  #::::E       EEEEE M:::::::::M     M:::::::::M RR::::R      R::::R
  #::::E             M::::::M:::M   M:::M::::::M   R:::R      R::::R
  #:::::EEEEEEEEEE   M:::::M M:::M M:::M M:::::M   R:::RRRRRR:::::R 
  #::::::::::::::E   M:::::M  M:::M:::M  M:::::M   R:::::::::::RR   
  #:::::EEEEEEEEEE   M:::::M   M:::::M   M:::::M   R:::RRRRRR::::R  
  #::::E             M:::::M    M:::M    M:::::M   R:::R      R::::R
  #::::E       EEEEE M:::::M     MMM     M:::::M   R:::R      R::::R
#E:::::EEEEEEEE::::E M:::::M             M:::::M   R:::R      R::::R
#::::::::::::::::::E M:::::M             M:::::M RR::::R      R::::R
#EEEEEEEEEEEEEEEEEEE MMMMMMM             MMMMMMM RRRRRRR      RRRRRR
 
# sudo yum update -y

In [1]:
#===============================================================================================================
# 1. CRÉATION D'UNE SESSION SPARK 
#===============================================================================================================

sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1678693160646_0013,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-12>

In [2]:
%%configure -f
{"conf": {
      "spark.pyspark.python": "python3",
      "spark.pyspark.virtualenv.enabled": "true",
      "spark.pyspark.virtualenv.type": "native",
      "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"
   }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
13,application_1678693160646_0014,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
13,application_1678693160646_0014,pyspark,idle,Link,Link,None,✔


In [3]:
#===============================================================================================================
# 2. MODULES DÉPENDANCES ET BIBLIOTHÈQUES (JupyterHub)
#===============================================================================================================

packages = ["sparkdl", "pillow", "pyspark", "pandas", "tensorflow", "pyarrow"]
for package in packages:
    sc.install_pypi_package(package)

#sc.install_pypi_package("pyarrow")
#sc.uninstall_package("pyarrow")    
#sc.list_packages()

from pyspark.sql.functions import element_at, split, pandas_udf, PandasUDFType, col
from pyspark.sql.functions import udf
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import VectorUDT, Vectors
from PIL import Image

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import img_to_array

import os
import numpy as np
import io
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281826065 sha256=0ccca0c4fca9565922b8f7c18ab7e7ed09eec02ca0226f9f4ede5b74a4f9e3c7
  Stored in directory: /mnt/tmp/pip-ephem-wheel-cache-pfmw40ms/wheels/07/fb/67/b9f2c0242d156eaa136b45ae4fd99d3e7c0ecc2acfd26f47b9
Successfully built pyspark






ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

python37-sagemaker-pyspark 1.4.2 requires pyspark==2.4.0, but you'll have pyspark 3.3.2 which is incompatible.


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver befo

In [4]:
#===============================================================================================================
# 3. CHARGEMENT DES IMAGES AU FORMAT BINAIRE
#===============================================================================================================

images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load("s3://fruitlabbucket/images/*")

images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))
print("Nombres d'images :", images.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------+-----------+
|path                                             |label      |
+-------------------------------------------------+-----------+
|s3://fruitlabbucket/images/apple_red_3/r1_67.jpg |apple_red_3|
|s3://fruitlabbucket/images/apple_red_3/r1_115.jpg|apple_red_3|
|s3://fruitlabbucket/images/apple_red_3/r1_267.jpg|apple_red_3|
|s3://fruitlabbucket/images/apple_red_3/r1_47.jpg |apple_red_3|
|s3://fruitlabbucket/images/apple_red_3/r1_123.jpg|apple_red_3|
+-------------------------------------------------+-----------+
only showing top 5 rows

None
Nombres d'images : 176

In [5]:
#===============================================================================================================
# 4. PRÉPARATION DU MODÈLE MOBILENETV2 
#===============================================================================================================

model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

new_model = Model(inputs=model.input, outputs=model.layers[-2].output)
brodcast_weights = sc.broadcast(new_model.get_weights())

def model_fn():
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

def preprocess(content):
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1678693160646_0014/container_1678693160646_0014_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [8]:
#===============================================================================================================
# 5. EXTRACTION DE FEATURES
#===============================================================================================================        

features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

features_df.write.mode("overwrite").parquet('s3://fruitlabbucket/resultMobileNet/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o257.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:104)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107

In [ ]:
#===============================================================================================================
# 6. RÉDUCTION DIMENSIONNELLE
#=============================================================================================================== 

data = spark.read.parquet('s3://fruitlabbucket/resultMobileNet/')
to_dense_vector = udf(lambda arr: Vectors.dense(arr), VectorUDT()) 
data = data.withColumn("features_dense", to_dense_vector("features")) 
pca = PCA(k=2, inputCol="features_dense", outputCol="pca_features") 
model = pca.fit(data) 
transformed_data = model.transform(data) 
result = transformed_data.select("pca_features") 
result.write.mode("overwrite").parquet("s3://fruitlabbucket/resultPCA/") 

def export_pca_features_to_csv(parquet_file_path, csv_file_path):
    result_pca = pd.read_parquet("s3://fruitlabbucket/resultPCA/", engine='pyarrow')
    pca_features_numeric = []  
    for row in result_pca['pca_features']:
        pca_features_numeric.append([float(x) for x in row['values']])

    pca_features_df = pd.DataFrame(pca_features_numeric, columns=['pca_feature_1', 'pca_feature_2'])
    pca_features_df.to_csv(csv_file_path, index=False) 

export_pca_features_to_csv("s3://fruitlabbucket/resultPCA/", "s3://fruitlabbucket/resultPCA/resultPCA.csv")     

In [ ]:
#===============================================================================================================
# 7. VISUALISATIONS DES RÉSULTATS
#=============================================================================================================== 

df1 = pd.read_parquet("s3://fruitlabbucket/resultMobileNet/", engine='pyarrow')
df2 = pd.read_csv("s3://fruitlabbucket/resultPCA/resultPCA.csv") 
df = pd.concat([df1, df2], axis=1)
df.to_csv('s3://fruitlabbucket/matrice.csv', index=False) 
df.head()